<a href="https://colab.research.google.com/github/hisunnytang/HaloEGNN/blob/main/notebooks/HaloEGNN_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install torchdiffeq
!pip install gdown
!git clone https://github.com/hisunnytang/HaloEGNN.git

In [2]:
!pip install -e HaloEGNN

Obtaining file:///content/HaloEGNN
  Running setup.py develop for HaloFlows


# Check the resources allocated

In [3]:
!nvidia-smi

Sun Apr  3 21:07:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:            0
CPU MHz:             2299.998
BogoMIPS:            4599.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs 

In [5]:
!cp /content/drive/MyDrive/HaloEGNN_sample_data/subhalo_columns.npy .
!cp /content/drive/MyDrive/HaloEGNN_sample_data/preprocessed_data.zip .

In [ ]:
# cannot directly use the cli for now, but you can still download this, and upload it back to colab
#!gdown https://drive.google.com/uc?id=1hpaQjfSi_TfdbmCEKf2PeSkDAlBX3jBV 

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1hpaQjfSi_TfdbmCEKf2PeSkDAlBX3jBV 



In [6]:
%%capture
!unzip preprocessed_data.zip

# Restart here for the system to see the package

In [1]:
from HaloEGNNFlows.train import *

In [ ]:
# import numpy as np
# import torch
# import glob
# import pandas as pd
# import ipywidgets as widgets
# import seaborn as sns
# import pickle
# from HaloEGNNFlows.EGNNFlows.models import get_model
# from HaloEGNNFlows.EGNNFlows.datasets import \
# (
#  ProgenitorDataset,
#  find_closest_redshift_slice, 
#  prepare_input_data
#  )

# from HaloEGNNFlows.EGNNFlows.viz.utils import compute_metric_features
# from HaloEGNNFlows.EGNNFlows.flows.utils import assert_correctly_masked
# from HaloEGNNFlows.EGNNFlows.flow_forward import  flow_forward
# import re
# from collections import OrderedDict

In [ ]:
# from HaloEGNN.train import train_loop, prepare_filelist_and_transformer, get_model

In [ ]:
!rm TNG300_preprocessed_data/prog_sublink_1849472.npy

In [8]:
preprocessed_loc = 'TNG300_preprocessed_data'
# Prepare file list
filelist = sorted(glob.glob(f"{preprocessed_loc}/prog_sublink_*.npy"))
# Prepare the columns names
full_columns_names = np.load(f"{preprocessed_loc}/subhalo_columns.npy")

# The "scalar" feature bounded to the progenitors
feature_columns = ["SubhaloMass", "SubhaloMergeRedshift"]
# The "positional" feature bounded to the progenitors
position_columns = ["SubhaloPos_0", "SubhaloPos_1", "SubhaloPos_2"]

# initialize the list condtional columns at redshift 0
condition_columns = [
    "SubhaloBHMass",
    "SubhaloBHMdot",
    "SubhaloGasMetallicity",
    "SubhaloStarMetallicity",
    "SubhaloMass",
    "DMFrac",
    "GasFrac",
    "StarWindFrac",
    "BHFrac",
    "SubhaloSFR",
    "SubhaloVmax",
    "SubhaloVelDisp",
]

In [6]:
failed = []
for f in filelist:
  try:
    np.load(f)
  except:
    print(f)
    failed.append(f)

TNG300_preprocessed_data/prog_sublink_1849472.npy


In [9]:
valid_files, condition_normalizer = prepare_filelist_and_transformer(
    filelist_npy=None,
    transformer_pkl=None,
    filelist=filelist,
    condition_columns=condition_columns,
    full_columns_names=full_columns_names,
    max_progenitors=20,
    initial_slice=0,
    final_slice=1,
    batch_size=512,
    num_workers=4,
    transform_type = "quantile"
)

In [10]:
import os
os.environ["PBS_JOBID"] = '040322'

In [ ]:
max_epochs = 1000
ode_reg    = 0.01

train_loop(
    valid_files,
    max_epochs,
    condition_normalizer,
    ode_reg,
    # The "scalar" feature bounded to the progenitors
    feature_columns,
    # The "positional" feature bounded to the progenitors
    position_columns,
    # initialize the list condtional columns at redshift 0
    condition_columns,
    full_columns_names,
    restart_path=None,
    batch_size=512,
    lr=1e-3,
    patience=3,
)

  0%|          | 0/119 [00:00<?, ?it/s]

initializing actnorm layer on device = cuda:0


100%|██████████| 15/15 [01:25<00:00,  5.70s/it, val_loss=92.6, nll=91.3]


Checkpoint: log_dir/040322/egnn_0_val=92.603.pt
Epoch 0: train loss = 103.70; val loss = 92.60


100%|██████████| 15/15 [01:45<00:00,  7.05s/it, val_loss=90.4, nll=89]


Checkpoint: log_dir/040322/egnn_1_val=90.398.pt
Epoch 1: train loss = 89.29; val loss = 90.40


 28%|██▊       | 33/119 [14:27<33:44, 23.54s/it, train_loss=87.9, nll=86.5]

In [4]:
ckpt_path = "egnn_16_val=74.837.pt"

In [5]:
# Load the checkpoint path
ckpt = torch.load(ckpt_path)

n_dims = len(position_columns)
in_node_nf = len(feature_columns)
context_node_nf = len(condition_columns)

valid_files, condition_normalizer = prepare_filelist_and_transformer(
    filelist_npy=None,
    transformer_pkl=None,
    filelist=filelist,
    condition_columns=condition_columns,
    full_columns_names=full_columns_names,
    max_progenitors=20,
    initial_slice=0,
    final_slice=1,
    batch_size=512,
    num_workers=4,
    transform_type = "quantile"
)

# TODO: hyperparameters should also be stored in the ckpt
# INITIALIZE the model
# Prepare Models and Priors/ Optimizer/ LR scheduler
prior, flow = get_model(
    in_node_nf=in_node_nf,  # Number of Features to fit (i.e. Progenitor Halo Mass)
    dynamics_in_node_nf=1,  # Use Time as additional Feature
    context_node_nf=context_node_nf,  # Number of Conditional Features
    n_dims=n_dims,  # Number of "Equivariant" Dimension
)
optim = torch.optim.AdamW(flow.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim, verbose=1, mode="min", min_lr=1e-8, patience=3
)
ode_regularization = 0.01
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
# Load state dict from last epoch
current_epoch = ckpt["epoch"]
flow.load_state_dict(ckpt["model_state_dict"])
optim.load_state_dict(ckpt["optimizer_state_dict"])
scheduler.load_state_dict(ckpt["scheduler_state_dict"])
# train_history
train_loss = ckpt["loss"]
val_loss = ckpt["val_loss"]
max_epochs = len(train_loss)

In [9]:
dirname = os.path.dirname(ckpt_path)
# condition_normalizer = pickle.load(open(os.path.join(dirname, "scaler.pkl"), "rb"))
log_path = create_log_directory("log_dir")

In [10]:
# Prepare dataloaders
dl_train, dl_val, dl_test = prepare_dataloaders(
    valid_files,
    condition_columns,
    full_columns_names,
    max_progenitors=20,
    initial_slice=0,
    final_slice=1,
    batch_size=512,
    num_workers=4,
    random_seed=42,
    train_test_split=[0.8, 0.1, 0.1],
    shuffle_train=True,
    distributed=False,
)

In [11]:
zidx = None

In [ ]:
for i in range(current_epoch, max_epochs):
  loss = train_step(
      flow,
      prior,
      optim,
      dl_train,
      condition_normalizer,
      device=device,
      ode_regularization=ode_regularization,
      transform_input=transform_z_to_scale(zidx),
  )
  val = val_step(
      flow,
      prior,
      dl_val,
      condition_normalizer,
      device=device,
      ode_regularization=ode_regularization,
      transform_input=transform_z_to_scale(zidx),
  )

  print(f"Epoch {i}: train loss = {loss.item():.2f}; val loss = {val.item():.2f}")
  train_loss[i] = loss
  val_loss[i] = val
  scheduler.step(val)
  checkpoint_model(
      flow, scheduler, optim, train_loss, val_loss, epoch=i, log_path=log_path
  )

100%|██████████| 15/15 [03:01<00:00, 12.07s/it, val_loss=74.9, nll=73]


Epoch 16: train loss = 74.78; val loss = 74.89
Checkpoint: log_dir/033022/egnn_16_val=74.894.pt


100%|██████████| 15/15 [02:36<00:00, 10.43s/it, val_loss=75, nll=73.1]


Epoch 17: train loss = 74.64; val loss = 74.97
Checkpoint: log_dir/033022/egnn_17_val=74.971.pt


100%|██████████| 15/15 [03:00<00:00, 12.04s/it, val_loss=74, nll=72]


...

100%|██████████| 15/15 [02:44<00:00, 10.98s/it, val_loss=72.8, nll=70.7]


Epoch 27: train loss = 72.27; val loss = 72.85
Checkpoint: log_dir/033022/egnn_27_val=72.848.pt


100%|██████████| 15/15 [03:00<00:00, 12.04s/it, val_loss=72.7, nll=70.5]


Epoch 28: train loss = 72.09; val loss = 72.70
Checkpoint: log_dir/033022/egnn_28_val=72.696.pt


 73%|███████▎  | 87/119 [57:20<21:33, 40.41s/it, train_loss=72.1, nll=70]